* Good features still allow you to solove problems nore elegantly while usign fewer resources. For instance, it would be ridiculous to solve the problem of reading a clock face using a convolutional neural network

* Good features let you sovle a problem with far less data. The ability of deep learning models to learn feautres on their own relies on having lots of training data available; if you have only a few samples. then the information value in their features becomes cirtical.

**Overfitting and underfitting**

* **Optimization** refers to the process of adusting a model to get the best performance possible on the training data(the learning in machine learning) 

* **Generalization** refers to how well the trained model performs on the data it has never seen before. The goal of the game is to get good geralization, of course, but you dont control generalization; you can only adjust the model based on its training data.

**Reducing the network's size**

* the simplest way to prevent overfitting is to reduce the size of the model: the number of learnable parameters in the model(which is determined by the number of layers and the number of units per layer)
    * a model with more parameters has more *memorization capacity* and thererfore can easily learn a perfect dictionary-like mapping between training samples and their targeets

* Deep learning models tend to be good at fitting to the training data, but the real challenge is generalization, not fitting.


**Adding weight regularization**

* a simple model in this context is a model where the distirbution of parameter values has less entropy. Thus a common way to mitigate overfitting is to put constraints on the complexity of a network bu forcing its weights to take only small values, which makes the distribution of weight values more *regular*. This is called weight reuglarization
  * L1 regularization -- the cost added is proportional to the absolute value of the weigh coefficients(the L1 norm of the weights).
  * L2 regularization --the cost added is proportional to the square of the value of the *weight cofficients**. L2 regularization is also called weigh decay in the context of neural networks

**Adding dropout**

## Problem

**Whate is rmsprop optimizer?**

**ROC curve**

* An ROC curve(receiver operating characteristic curve) is a graph showing the performance of a classification model at all classififcation thresholds. This curve plots two parameters:
    * True positive rage
    * False Positive rate

**AUC: Area Under the ROC Curve**

* AUC stands for "Area under the ROC Curve." That is, AUC measures the entire two-dimensional area underneath the entire ROC curve(thin integral calculus) from (0,0) to (1,1).

**This key characteristic gives convnets two interesting properties:**
* The patterns they learn are translation invarint. After learning a certain pattern in the lower-right corner of a picture, a convnet can recongnize it anywhere: for example, in the upper-left corner. Adensely connected network would have to learn the pattern anew if it appeared at a new location. This makes convnets data efficient weh processing images(because teh visual world is fundamentally translation invariant): they need fewer training samples to leaern representations that have generalization power.
* They can learn spartial hierarchies of patterns. Afirst convolution layer will learn small local patterns such as edges, a second convolution layer will learn larger patterns made of the features of the first layers, and so on. This allows convnets to efficiently learn increasingly complex and abstrac visual concepts(because the visual world is fundamentally spatially hierarchical).

**Convolution operate over 3D tensors, called feature maps**

**Note that the output width and height may differ from the input width and height. They may differ for two reason:**
* Border effects, which can be countered by padding teh input feature map
* The use of strides

In Conv2D layers, paddding is configurable vai the padding argument, which takes two values: "valid", which means no padding(only valid window laocations wil be used);
and "same", which means "pad in such a way as to have an output with the same width and height as the input." The padding argument defaults to "valid".

**Understanding convolution strides**

The other factor that can influence output size is the notion of strides. The description of convolution so far has assumed that the center tiles of the convolution windows are all contiguous. But the distance between two successive windows is a parameter of the convolution, called its stride, which defaults to 1. It's possible to have strided convolutions: convolutions with a stride higher than 1. 

**The max-pooling operation**

In the convnet exaple, you may have noticed that the size of the feature maps is halved after every MaxPooling2dD layer. For instance, before the first MaxPooling2D layers, the feature map is 26x26, but the max-pooling operation halves it to 13x13. That's the role of max pooling: to aggressively downsample feature maps, muuch like strided convolutions.

# Using a pretrained convnet

## Feature extraction

Freature extraction consists of using the representations learned by a previous network to extract interesting features from new samples. These features are then run throuhg a new classifier, which is trained from scratch

In teh case of convnets, feature extraction consists of taking the convolutional base of a previously trained network, runing the new data through it, and training a new classifier on top of the output

if your new dataset differs a lot from the dataset on which the original model was trained, you may be better off using only the first few layers of the model to do feature extraction, rather than using the entire convolutional base.

**Finte-tuning**

Finte-tuning consists of unfreezing a few of the top layers of a frozen model base used for feature extraction, and jointly training both newly added part of the model and these top layers. This is called fine-tunning because it slightly adjusts the more abstract representations of th model being reused, in order to make them more relevant for the problem at hand.

In this case, because the ImageNet class set contrains multiplue dog and cate dlassed, it's likely to be beneficial to reuse the information contained in the densely connected layers of the original model. But we'll choose  not to, in order to cover the more general case where the class set of the new problem doesn't overlap the class set of the riginal model. Let's put thi in practice by using the convolutional base of the VGG16 network, trained on ImageNet, to extract interesting features from cat and dog images, and then train a dags-versus-cats classifier on top of these features.

In [1]:
from keras.applications import VGG16

Using TensorFlow backend.


In [2]:
conv_base = VGG16(weights = 'imagenet', include_top = False, input_shape=(150,150,3))

58892288/58889256 [==============================] - 1s 0us/step


* weights specifies the weight checkpoint from which to initialize the model.
* include_top refers to including( or not) the densely connected classifier on top of the network. By default, this densely connected classifier corresponeds to the 1,000 classes from ImageNet. Because you intend to use your own densely connected classifier( with only two classes: cat and dog), you dont need included it.
* input_shape is the shape of the image tensors that you will feed to the network. This argument is purely optional: if you dont passit, the network will be able to process inputs of any size.

In [3]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

At this point, there are two ways you could proceed:
* Running the convolutional base over your dataset, recording its output to a Numpy array on disk, and then using this data as input to a standalone, densely connected classifier similar to those you saw in part 1 of this book. This solution is fast and cheap to run, because it only requires running the convolutional base once for every input image, and the convolutional base is by far the most expensive part of the pipeline. But for the dame reason, this technique wont allow you to use data augmentation.
* Extending the model you have(conv_base) by adding Dense layers on top, and running the whole thing end to end on the input data. This will allow you to use data agumentation, because every input image goes through the convolutional base every time it's senn by the model. But for the same reason, this technique is far more expensive than the first.

**Fast Feature extraction without data augmentation**

In [1]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator


In [2]:
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_feature(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4,4,512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size = (150,150),
        batch_size=batch_size,
        class_model='binary'
    )
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i*batch_size:(i+1)*batch_size] = features_batch
        labels[i*batch_size:(i+1)*batch_size] = labels_batch
        i+=1
        if i*batch_size >= sample_count:
            break
    return features, labels


### Feature extraction with data augmentation

Extending the conv_base model and running it end to end on the input
* Note this technique is so expensive that you should only attempt it if you have access to a GPU --it's absolutely intractable on CPU.

Before you compile and train the model, it's very important to freeze the convolutional base. Frreezing a layer or set of layers means preventing their weights from being updated during training. If you do not do this, then the representations that were previously learned by the convolutional base will be modified during training. Because the Dense layers on  top are randomly initialized, very large weight updates would be propagated through the network, effectively destroying the representations previously learned.

With this setup, only the weights from the two Dense layers that you added will be trained. That's a total of four weight tensor: two per layer(the main weight matrix and the bias vector). Note that in order for these changes to take effect, you must first compile the model. if you ever modify weight trainability after compilation, you should then recompile the model, or these changes will be ignored